### Introduction
In active learning (AL), we use a machine learning (ML) model as a surrogate for a more computationally expensive method.  Let's say we want to dock millions of molecules, but have limited computational resources.  We could sample a subset of the molecules we want to dock and dock the subset.  The chemical structures and docking scores for the subset could then be used to build an ML model to predict the docking scores for the entire set of molecules. In AL, we perform multiple cycles of this prediction and sampling workflow. Throughout the process, we want to balance two parameters.
- Exploration - efficiently search the chemical space and identify the most promising regions
- Explotation - focus the search on the most interesting regions of chemical space

This process is illustrated in the figure below.  The two red boxes represent the **oracle** that performs the more expensive calculations.
<br>
<center><img src="images/active_learning.png" alt="active learning figure" width="800"/></center>

1. Begin with a pool of M molecules
2. Sample N molecules from the pool
3. Perform the computationally expensive calculations on the subset of N molecules.  If our objective is docking, we dock the N molecules.
4. The chemical structures and docking scores for the N molecules are used to build an ML model
5. The model from step 4 ised to predict values for the M molecules from step 1
6. The predictions from the previous step are used to select another set of N molecules.  There are several ways to do this.  One of the simplest is a **greedy** search where we select the N best scoring molecules.  Alternately, we can employ strategies that use the uncertainty in the predictions to direct exploration.
7. Perform the computationally expensive calculations on the molecules selected in step 6. If our objective is docking, we would dock the N molecules.
8. The results from step 7 are combined with the results from step 3 and the model is retrained.
Steps 4 through 8 can be repeated multiple times

In the example below, we use AL to dock a set of 100K molecules.  To make this exercise more time efficient, we'll look up the activity rather performing the docking. The code below uses modAL, an open source library for active learning. Modal provides several different AL strategies.

### Installation

In [ ]:
!pip install pandas numpy seaborn useful_rdkit_utils modAL tqdm scikit-learn modAL

### Setup

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import useful_rdkit_utils as uru
from modAL.models import ActiveLearner
from sklearn.ensemble import RandomForestClassifier
from tqdm.auto import tqdm

Enable Pandas **progress_apply**

In [ ]:
tqdm.pandas()

Read data from ["Traversing Chemical Space with Active Deep Learning"](https://chemrxiv.org/engage/chemrxiv/article-details/654a603348dad23120461847) by Derek van Tilborg and Francesca Grisoni. 

The data is formatted with SMILES and 1 or 0 inidicating active or inactive. 
```
smiles,y
COc1cc(/C=N/NC(=O)C(=O)NCC2CCCO2)ccc1O,0
CC1CCCCC12NC(=O)N(CC(=O)Nc1ccc(N3CCOCC3)cc1)C2=O,0
C[NH+]1CCCC(OC(=O)c2cccc(Cl)c2)C1,0
CCOc1ccc(C(=O)NCC(=O)OCC(=O)N2CCCC2)cc1,0
```
After reading the data w generate fingerprints as descriptors. The function **uru.smi2numpy_fp** takes SMILES as input and returns a fingerprint as a numpy array.

In [ ]:
df = pd.read_csv("data/screen.csv")
df['fp'] = df.smiles.progress_apply(uru.smi2numpy_fp)

Convert the data to numpy arrays

In [ ]:
%%time
X_pool = np.stack(df.fp.values)
y_pool = df.y.values

### Initial Model
Here's where we define an oracle to return the results of our calculation.  In this case, we're just looking up a value.  In practice an oracle might perform docking calcuations or something else that's more compute itensive. The notebook **active_shape_search.ipynb** has a complete implementation of an oracle. 

In [ ]:
class Oracle:
    def __init__(self, df):
        self.df = df

    def get_values(self, idx_list):
        return df.y.values[idx_list]

Select a set of molecules to build and initial model.  In this case, we'll randomly select 100 molecules and use this set of 100 to build an ML model. 

In [ ]:
n_initial = 100
initial_list = np.random.choice(range(len(df)), size=n_initial, replace=False)

Select 100 examples to build the initial model

In [ ]:
X_training = X_pool[initial_list]
y_training = y_pool[initial_list]

### Active Learning
Define an ActiveLearner.  This class holds the ML model used to preform the active learning. In this case we'll use a RandomForestClassifier from scikit_learn as our ML model. 

In [ ]:
learner = ActiveLearner(
    estimator=RandomForestClassifier(),
    X_training=X_training, y_training=y_training
)

Instantiate an oracle.  As mentioned above, this is a simple lookup. 

In [ ]:
oracle = Oracle(df)

Run 10 cycles of active learning. We'll print out the number of active molecules we've found at each iteration.

In [ ]:
# Define a list to keep track of the molecules we've selected
pick_list = initial_list
# How many molecules we will select at each iteration
n_instances = 100
for i in range(0, 10):
    # Use the model to select the next set of molecules
    query_idx, query_inst = learner.query(X_pool, n_instances=n_instances)
    # Use the oracle to look up the value
    y_new = oracle.get_values(query_idx)
    # Use the values from the oracle to update the model
    learner.teach(X_pool[query_idx], y_pool[query_idx])
    # Add the picks to pick_list
    pick_list = np.append(pick_list, query_idx)
    # How many active molecules have we found
    print(sum(y_pool[pick_list]))

### Compare With a Random Baseline
That looks pretty good, but we should compare with a random baseline.  Let's select 1,000 random molecules and see how many actives we find.

In [ ]:
random_hit_count_list = []
for i in range(0, 10):
    random_list = np.random.choice(range(len(df)), size=1000, replace=False)
    random_hit_count_list.append(sum(df.y.values[random_list]))

Look at the number of active molecules we found with a random search.

In [ ]:
random_hit_count_list

Now lets run 10 active learning trials.  To do this, we'll write a function that encapsulates the active learning code we wrote above.

In [ ]:
def run_active_learning(X, y, oracle, num_cycles=10):
    initial_list = np.random.choice(range(len(df)), size=n_initial, replace=False)
    pick_list = initial_list
    learner = ActiveLearner(
        estimator=RandomForestClassifier(),
        X_training=X_training, y_training=y_training
    )
    for i in tqdm(range(0, num_cycles)):
        query_idx, query_inst = learner.query(X_pool, n_instances=n_instances)
        y_new = oracle.get_values(query_idx)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        pick_list = np.append(pick_list, query_idx)
    return sum(y[pick_list])

In [ ]:
active_learning_hit_count_list = []
for i in range(0, 10):
    num_hits = run_active_learning(X_pool, y_pool, oracle)
    active_learning_hit_count_list.append(num_hits)

Look at the number of hits we found with active learning.  Note that it's a lot more than what we found with a random search.

In [ ]:
active_learning_hit_count_list

Let's make a boxplot to compare the random and active learning searches.   This will be a lot easier if we put the data into a dataframe.

In [ ]:
random_df = pd.DataFrame(random_hit_count_list)
random_df.columns = ["count"]
random_df['category'] = "random"
active_df = pd.DataFrame(active_learning_hit_count_list)
active_df.columns = ["count"]
active_df['category'] = "active"
plot_df = pd.concat([random_df, active_df])

Now make the boxplot

In [ ]:
sns.boxplot(data=plot_df, x="category", y="count");

Calculate the enrichment ratio for active learning vs random

In [ ]:
np.mean(active_learning_hit_count_list) / np.mean(random_hit_count_list)